### pull pybaseball data

In [2]:
import pandas as pd 
from pybaseball import statcast
from pybaseball import statcast_batter
from pybaseball import cache
import os
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Active/Swing Decisons')
cache.enable()

In [3]:
df = statcast('2018-01-01', '2024-12-30')

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


 14%|█▍        | 214/1520 [04:21<08:34,  2.54it/s]  c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 14%|█▍        | 215/1520 [04:27<36:36,  1.68s/it]c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 14%|█▍        | 216/1520 [04:27<30:49,  1.42s/it]c:\Users\dalto\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in

In [4]:
df_keep = df[['game_date','pitch_type', 'pitch_name', 'player_name', 'pitcher', 'batter', 'description', 'zone', 'estimated_woba_using_speedangle', 'stand', 'p_throws', 'type', 'balls', 'strikes',
              'plate_x','plate_z', 'sz_top', 'sz_bot']]
df_keep = df_keep[df_keep['pitch_type'].notna()]
df_keep = df_keep.rename(columns={'estimated_woba_using_speedangle': 'xwoba', 'stand': 'b_stands', 'plate_x': 'plate_horz', 'plate_z': 'plate_vert'})

In [5]:
batter = pd.read_csv('./data/batter_data/fg_00_24.csv')
batter = batter[['Name', 'MLBAMID']]
batter = batter.reset_index()
batter = batter.drop(columns=['index'])

In [6]:
df_keep = df_keep.merge(batter, how='left',left_on='batter', right_on='MLBAMID')

In [7]:
df_keep = df_keep.drop_duplicates()

In [8]:
# Move 'Name' column to be the 4th column and drop 'MLBAMID'
cols = list(df_keep.columns)
cols.remove('Name')
cols.remove('pitcher')
cols.remove('batter')
cols = cols[:3] + ['Name'] + cols[3:]
df_keep = df_keep[cols]

In [9]:
df_keep = df_keep.rename(columns={'player_name': 'pitcher', 'Name':'batter'})

In [10]:
import unicodedata
def clean_names(df, name_col):
    # Make a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Convert to lowercase and remove leading/trailing whitespace
    df[name_col] = df[name_col].str.lower().str.strip()
    
    # Remove accents
    df[name_col] = df[name_col].apply(lambda x: ''.join(
        c for c in unicodedata.normalize('NFKD', str(x))
        if not unicodedata.combining(c)
    ))
    
    # Keep only letters and replace multiple spaces with single space
    df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)
    
    return df

<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dalto\AppData\Local\Temp\ipykernel_18592\2225678073.py:16: SyntaxWarning: invalid escape sequence '\s'
  df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)


In [11]:
df_keep['pitcher'] = df_keep['pitcher'].str.split(', ').str[::-1].str.join(' ')
df_keep = clean_names(df_keep, 'pitcher')
df_keep = clean_names(df_keep, 'batter')

In [13]:
df_keep.to_csv('./data/swing_data/pitch_18_24.csv')